In [1]:
import pandas as pd
frame = pd.read_table("sampling_frame.tsv")
frame.head()

,user_name,home_wiki,global_edits,wiki,edits
0,! Bikkit !,dewiki,1658.6,dewiki,1435.0
1,! Bikkit !,dewiki,1658.6,enwiki,190.0
2,! Bikkit !,dewiki,1658.6,commonswiki,32.0
3,! Bikkit !,dewiki,1658.6,frwiki,1.0
4,! Bikkit !,dewiki,1658.6,wikidatawiki,0.6


In [2]:
per_wiki = frame[["user_name", "wiki", "edits"]]
per_wiki = per_wiki.pivot("user_name", "wiki")
per_wiki.columns = per_wiki.columns.droplevel()
per_wiki.head()

wiki,abwiki,acewiki,adywiki,afwiki,afwikibooks,afwikiquote,afwiktionary,akwiki,alswiki,amwiki,...,zh_yuewiki,zhwiki,zhwikibooks,zhwikinews,zhwikiquote,zhwikisource,zhwikivoyage,zhwiktionary,zuwiki,zuwiktionary
user_name,,,,,,,,,,,,,,,,,,,,,
! Bikkit !,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!KrzysiekBu!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!Manihiki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!Silent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
!minmi73!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
global_data = frame[["user_name", "home_wiki", "global_edits"]]
grouped = global_data.groupby("user_name")
global_data = grouped.first().reset_index()
global_data.head()

,user_name,home_wiki,global_edits
0,! Bikkit !,dewiki,1658.6
1,!KrzysiekBu!,plwiki,2463.6
2,!Manihiki,itwikibooks,648.0
3,!Silent,ptwiki,26321.5
4,!minmi73!,jawiki,64.0


In [4]:
frame = global_data.join(per_wiki, on = "user_name")
frame.head()

,user_name,home_wiki,global_edits,abwiki,acewiki,adywiki,afwiki,afwikibooks,afwikiquote,afwiktionary,...,zh_yuewiki,zhwiki,zhwikibooks,zhwikinews,zhwikiquote,zhwikisource,zhwikivoyage,zhwiktionary,zuwiki,zuwiktionary
0,! Bikkit !,dewiki,1658.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,!KrzysiekBu!,plwiki,2463.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,!Manihiki,itwikibooks,648.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,!Silent,ptwiki,26321.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,!minmi73!,jawiki,64.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
def sample_stratum(stratum_query, sample_size):
    """
    returns a tuple consisting of (1) the population count for that subgroup and
    (2) a list of the sampled users in MassMessage format 
    """
    stratum = frame.query(stratum_query)
    stratum_size = len(stratum)
    sample = stratum.sample(n = sample_size)[["user_name", "home_wiki"]]
    sample_list = [
        "{}@{}".format(i.user_name, i.home_wiki) 
        for i in sample.itertuples(index = False, name = "user")
    ]
    return (sample_size, sample_list)

In [61]:
sample_stratum("global_edits >= 600 & home_wiki.isin(['enwiktionary', 'frwiktionary'])", 10)

(10,
 ['Kiminou1@frwiktionary',
  'Mulder1982@enwiktionary',
  'Supreme assis@frwiktionary',
  'Unsui@frwiktionary',
  'Urhixidur@frwiktionary',
  'Lincoln1001@enwiktionary',
  'Fumiko Take@enwiktionary',
  'Hongthay@enwiktionary',
  'Fletcherjp@enwiktionary',
  'Useigor@enwiktionary'])

In [65]:
import pywikibot as pwb

In [73]:
site = pwb.Site("test", "wikipedia")
page = pwb.Page(site, u"Main Page")
page.text

ERROR: Traceback (most recent call last):
  File "/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pywikibot/data/api.py", line 1557, in submit
    body=body, headers=headers)
  File "/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pywikibot/tools/__init__.py", line 1105, in wrapper
    return obj(*__args, **__kw)
  File "/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pywikibot/comms/http.py", line 279, in request
    r = fetch(baseuri, method, body, headers, **kwargs)
  File "/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pywikibot/comms/http.py", line 381, in fetch
    error_handling_callback(request)
  File "/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pywikibot/comms/http.py", line 297, in error_handling_callback
    raise request.data
  File "/home/neilpquinn-wmf/venv/lib/python3.4/site-packages/pywikibot/comms/threadedhttp.py", line 217, in request
    self.add_credentials(*config.authenticate[authority])
TypeError: add_credentials() takes from 3

KeyboardInterrupt: 